一组预测器称为集成, 这种技术也被称为集成学习, 而一个集成学习算法则被称为集成方法。

# 7.1 投票分类器

已经训练好了一些分类器，要创建出一个更好的分类器，最简单的方法就是聚合每个分类器的预测，然后将得票最多的结果作为预测类别  
这种大多数投票分类器被称为硬投票分类器

创建并训练一个投票分类器，由三种不同的分类器组成，训练集是卫星数据集

In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=10000, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [2]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8753333333333333
RandomForestClassifier 0.988
SVC 0.9896666666666667
VotingClassifier 0.989


# 7.2 bagging和pasting

bagging: 每个预测器使用的算法相同，但在不同的训练集随机子集上进行训练  
pasting: 采样时样本不放回，bagging允许训练实例被同一个预测器多次采样

与直接在原始训练集上训练的单个预测器相比，集成的偏差相近，但是方差更低

## 7.2.1 Scikit-Learn中的bagging和pasting

In [3]:
# 训练一个包含500个决策树分类器的集成
# 每次从训练集随机采样100个训练实例，然后放回
# 如果使用pasting，只需设置bootstrap=False
# n_jobs指示多少CPU内核进行训练和预测(-1表示所有可用)

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500,
    max_samples=100, 
    bootstrap=True, 
    # n_jobs=-1         !会报错
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

## 7.2.2 包外评估

未被采样的训练实例称为包外(oob)实例  
设置oob_score=True在训练结束自动进行包外评估

In [4]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500,
    max_samples=100, 
    bootstrap=True, 
    # n_jobs=-1         !会报错
    oob_score=True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.974

In [5]:
# 验证
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9733333333333334

测试集上获得96.86%的准确率，与oob_score接近

In [6]:
# 每个训练实例的包外决策函数也可以通过变量oob_decision_function_获得

bag_clf.oob_decision_function_

array([[0.32244898, 0.67755102],
       [0.97160243, 0.02839757],
       [0.        , 1.        ],
       ...,
       [0.95519348, 0.04480652],
       [0.98985801, 0.01014199],
       [0.98170732, 0.01829268]])

# 7.3 随机补丁和随机子空间

BaggingClassifier类也支持对特征进行采样  
采样由两个超参数控制:max_features和bootstrap_features  
对高维输入(例如图像)特别有用

# 7.4 随机森林

随机森林是决策树的集合  

In [7]:
# 训练一个拥有500棵树的随机森林分类器

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

随机森林在树的生长上引入了更多的随机性:分裂节点时不再是搜索最好的特征，而是在一个随机生成的特征子集里搜索最好的特征

## 7.4.1 极端随机数

对每个特征使用随机阈值，而不是搜索得出的最佳阈值，则可能让决策树生长得更加随机  
使用Scikit-learn的ExtraTreesClassifier类

## 7.4.2 特征重要性

In [8]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)


sepal length (cm) 0.10298648310206393
sepal width (cm) 0.02822564894689312
petal length (cm) 0.43100758506750814
petal width (cm) 0.4377802828835348


# 7.5 提升法

提升法(boosting)是指可以将几个弱学习器结合成一个强学习器的任意集成方法

## 7.5.1 AdaBoost

新预测器对前序进行纠正的方法之一就是更多地关注前序欠拟合地训练实例，从而使新的预测器不断地越来越专注于难缠的问题

Scikit-Learn的AdaBoostClassifier训练一个AdaBoost分类器，它基于200个单层决策树

In [9]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    algorithm="SAMME.R",    # SAMME即等同于AdaBoost, SAMME.R是一种变体, 它依赖的是类概率而不是类预测, 通常表现更好
    learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

## 7.5.2 梯度提升

让新的预测器针对前一个预测器的残差进行拟合

梯度提升回归树(GBRT)

In [10]:
# 实例
# 首先在训练集上拟合一个DecisionTreeRegressor

from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2)

In [11]:
# 针对第一个预测器的残差，训练第二个DecisionTreeRegressor

y2 = y_train - tree_reg1.predict(X_train)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X_train, y2)

DecisionTreeRegressor(max_depth=2)

In [12]:
# 针对第二个预测器的残差，训练第三个DecisionTreeRegressor

y3 = y2 - tree_reg2.predict(X_train)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X_train, y3)

DecisionTreeRegressor(max_depth=2)

In [13]:
# 将所有树的预测相加，从而对新实例进行预测

y_pred = sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3))

训练GBRT的简单方法

In [14]:
from sklearn.ensemble import GradientBoostingRegressor

# learning_rate对每棵树的贡献进行缩放，如果设为低值如0.1则需要更多的树来拟合训练集，但是预测的泛化效果通常更好
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)


要找到树的最佳数量，可以使用提前停止法  
使用staged_predict()方法，在训练的每一阶段都对集成的预测器返回一个迭代器，然后测量每个训练阶段的验证误差，从而找到树的最优数量，最后使用最优树数重新训练一个GBRT集成

In [15]:
import numpy as np
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)     # 初始120
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1    # 检索数组中最小值的位置，并返回其下标值

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)



GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [17]:
# 实现提前停止不一定需要先训练大量的树，然后再回头找最优的数字，还可以提前停止训练
# 设置warm_start=True，从而允许增量训练

# 验证误差连续5次迭代未改善，直接停止训练

gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

XGBoost  
Author: Tianqi Chen

In [20]:
import xgboost

xbg_reg = xgboost.XGBRegressor()
xbg_reg.fit(X_train, y_train)
y_pred = xbg_reg.predict(X_val)

In [21]:
# 自动提前停止

xbg_reg.fit(X_train, y_train, eval_set=[(X_val, y_pred)], early_stopping_rounds=2)
y_pred = xbg_reg.predict(X_val)

[0]	validation_0-rmse:0.35022
[1]	validation_0-rmse:0.24846
[2]	validation_0-rmse:0.17847
[3]	validation_0-rmse:0.13094
[4]	validation_0-rmse:0.09870
[5]	validation_0-rmse:0.07821
[6]	validation_0-rmse:0.06440
[7]	validation_0-rmse:0.05586
[8]	validation_0-rmse:0.05121
[9]	validation_0-rmse:0.04832
[10]	validation_0-rmse:0.04635
[11]	validation_0-rmse:0.04480
[12]	validation_0-rmse:0.04391
[13]	validation_0-rmse:0.04243


C:\Users\李子豪\AppData\Roaming\Python\Python310\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[14]	validation_0-rmse:0.04150
[15]	validation_0-rmse:0.03633
[16]	validation_0-rmse:0.03318
[17]	validation_0-rmse:0.03295
[18]	validation_0-rmse:0.03161
[19]	validation_0-rmse:0.03084
[20]	validation_0-rmse:0.03062
[21]	validation_0-rmse:0.03050
[22]	validation_0-rmse:0.02987
[23]	validation_0-rmse:0.02941
[24]	validation_0-rmse:0.02932
[25]	validation_0-rmse:0.02816
[26]	validation_0-rmse:0.02759
[27]	validation_0-rmse:0.02682
[28]	validation_0-rmse:0.02648
[29]	validation_0-rmse:0.02632
[30]	validation_0-rmse:0.02558
[31]	validation_0-rmse:0.02445
[32]	validation_0-rmse:0.02403
[33]	validation_0-rmse:0.02344
[34]	validation_0-rmse:0.02245
[35]	validation_0-rmse:0.02226
[36]	validation_0-rmse:0.02210
[37]	validation_0-rmse:0.02143
[38]	validation_0-rmse:0.02071
[39]	validation_0-rmse:0.01978
[40]	validation_0-rmse:0.01928
[41]	validation_0-rmse:0.01916
[42]	validation_0-rmse:0.01861
[43]	validation_0-rmse:0.01791
[44]	validation_0-rmse:0.01730
[45]	validation_0-rmse:0.01613
[46]	val

# 7.6 堆叠法

使用留存集  
1. 将训练集分为两个子集，第一个子集用来训练第一层的预测器
2. 第一层的预测器在第二个子集上进行预测
3. 对于留存集中的每个实例都有了三个预测值，使用这些预测值创建一个新的训练集，并保留目标值
4. 在这个新的训练集上训练混合器，让它学习根据第一层的预测来预测目标值